# AI Profile Comparison

Statistical comparison of AI profile performance.

In [ ]:
import polars as pl
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

from dicee_analysis import (
    load_games,
    compare_profiles,
    plot_profile_comparison,
    plot_win_rates,
    plot_bonus_rates,
)

In [ ]:
# Load data
RESULTS_PATH = "../../../results/games.ndjson"
games = load_games(RESULTS_PATH, progress=True)
print(f"Loaded {games.n_unique('game_id')} games")

## Profile Comparison Overview

In [ ]:
# Visual comparison with confidence intervals
fig = plot_profile_comparison(games, title="AI Profile Performance Comparison")
plt.show()

## Statistical Tests

Compare specific profile pairs using Welch's t-test.

In [ ]:
# Compare professor vs carmen
profiles = games["profile_id"].unique().to_list()

if len(profiles) >= 2:
    result = compare_profiles(games, profiles[0], profiles[1])
    print(f"Comparing {profiles[0]} vs {profiles[1]}:")
    print(f"  {result.conclusion}")
    print(f"  p-value: {result.p_value:.4f}")
    print(f"  Effect size: {result.effect_size:.3f} ({result.effect_interpretation})")

In [ ]:
# Pairwise comparisons
from itertools import combinations

print("Pairwise Profile Comparisons (Welch's t-test):\n")
print(f"{'Profile 1':<12} {'Profile 2':<12} {'p-value':<10} {'Effect':<8} {'Significant'}")
print("-" * 60)

for p1, p2 in combinations(profiles, 2):
    result = compare_profiles(games, p1, p2)
    sig = "Yes" if result.significant else "No"
    print(f"{p1:<12} {p2:<12} {result.p_value:<10.4f} {result.effect_size:<8.3f} {sig}")

## Win Rates (for multiplayer games)

In [ ]:
# Only meaningful if games have multiple players
n_players_per_game = games.group_by("game_id").agg(pl.len().alias("n"))["n"].mean()

if n_players_per_game > 1:
    fig = plot_win_rates(games, title="Win Rates by Profile")
    plt.show()
else:
    print("Single-player games - win rates not applicable")

## Upper Bonus Achievement

In [ ]:
fig = plot_bonus_rates(games, title="Upper Bonus Achievement Rate")
plt.show()